# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import nltk
nltk.download(['punkt','stopwords', 'wordnet'])

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pickle
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dengl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dengl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dengl\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
# load data from database
engine = create_engine('sqlite:///DisasterRespond.db')
df = pd.read_sql('cleaned_data',con = engine)


In [11]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
X = df['message'].values
y = df[df.columns[4:]]

In [15]:
def load_data():
    '''
    Load data from sqlite database and set X and y for modeling preparation
    
    OUTPUT
        - X: array of messages
        - y: target dataframe
    '''
    engine = create_engine('sqlite:///DisasterRespond.db')
    df = pd.read_sql('cleaned_data',con = engine)
    X = df['message'].values
    y = df[df.columns[4:]]
    return X, y

In [16]:
X

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name', ...,
       "Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families.",
       'Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.',
       'A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.'],
      dtype=object)

In [17]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [18]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [19]:
def tokenize(text):
    '''
        Processed text in string format, and output a list of words
        INPUT:
            -text: string, raw text data
        OUTPUT:
            -clean_tokens, list of processed words
    
    '''
    # get list of all urls using regex
    detected_urls = re.findall(url_regex,text)
    # replace each url in text strings with placeholder
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder')
    
    # Case Normalization
    text = text.lower() # convert to lowercase
    # remove puntuation characters
    text = re.sub(r"[^a-zA-Z0-9]", " ",text)
    
    # tokenize text
    tokens = word_tokenize(text)
    token_list =[]
    # remove stop words
    for tok in tokens:
        if tok not in stopwords.words("english"):
             token_list.append(tok)
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # iritate through each token
    clean_tokens = []
    for tok in token_list:
        # lemmatize and remove leading and tailing white space
        clean_tok = lemmatizer.lemmatize(tok).strip()
        
        clean_tokens.append(clean_tok)
    return clean_tokens


In [23]:
print(tokenize(X[3]))

['un', 'report', 'leogane', '80', '90', 'destroyed', 'hospital', 'st', 'croix', 'functioning', 'need', 'supply', 'desperately']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [20]:
# create the pipeline
parameters = 
pipeline = Pipeline([
        ('vect',CountVectorizer(tokenizer=tokenize)),
        ('tfidf',TfidfTransformer()),
        ('clf',MultiOutputClassifier(RandomForestClassifier(n_jobs=-1)))
]) # n_jobs=-1 means using all available processes / thread

In [21]:
pipeline.get_params()

{'clf': MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)),
 'clf__estimator': RandomForestClassifier(n_jobs=-1),
 'clf__estimator__bootstrap': True,
 'clf__estimator__ccp_alpha': 0.0,
 'clf__estimator__class_weight': None,
 'clf__estimator__criterion': 'gini',
 'clf__estimator__max_depth': None,
 'clf__estimator__max_features': 'auto',
 'clf__estimator__max_leaf_nodes': None,
 'clf__estimator__max_samples': None,
 'clf__estimator__min_impurity_decrease': 0.0,
 'clf__estimator__min_impurity_split': None,
 'clf__estimator__min_samples_leaf': 1,
 'clf__estimator__min_samples_split': 2,
 'clf__estimator__min_weight_fraction_leaf': 0.0,
 'clf__estimator__n_estimators': 100,
 'clf__estimator__n_jobs': -1,
 'clf__estimator__oob_score': False,
 'clf__estimator__random_state': None,
 'clf__estimator__verbose': 0,
 'clf__estimator__warm_start': False,
 'clf__n_jobs': None,
 'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x000001BF111E3D90>

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [24]:
# train test data split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.25, random_state=42)

# train classifier
pipeline.fit(X_train,y_train)
# predict test data
y_pred = pipeline.predict(X_test)

In [25]:
# calculate accuracy
accuracy = (y_pred == y_test).mean()

print("Accuracy:", accuracy)

Accuracy: related                   0.812023
request                   0.888923
offer                     0.984590
aid_related               0.763656
medical_help              0.912878
medical_products          0.941410
search_and_rescue         0.967043
security                  0.971926
military                  0.962313
child_alone               0.990998
water                     0.947818
food                      0.926915
shelter                   0.930882
clothing                  0.977724
money                     0.972078
missing_people            0.980928
refugees                  0.958193
death                     0.953311
other_aid                 0.859323
infrastructure_related    0.926457
transport                 0.947360
buildings                 0.944919
electricity               0.969027
tools                     0.982911
hospitals                 0.980928
shops                     0.986421
aid_centers               0.978944
other_infrastructure      0.947360
weather_re

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [26]:
print(classification_report(y_test,y_pred,target_names = df.columns[4:]))


                        precision    recall  f1-score   support

               related       0.83      0.94      0.88      4991
               request       0.81      0.49      0.61      1158
                 offer       0.27      0.15      0.19        80
           aid_related       0.72      0.70      0.71      2717
          medical_help       0.54      0.13      0.20       582
      medical_products       0.53      0.14      0.22       391
     search_and_rescue       0.43      0.15      0.22       206
              security       0.29      0.09      0.13       163
              military       0.50      0.17      0.25       247
           child_alone       0.33      0.26      0.29        47
                 water       0.75      0.40      0.52       465
                  food       0.80      0.53      0.64       792
               shelter       0.75      0.41      0.53       628
              clothing       0.49      0.21      0.29       145
                 money       0.47      

C:\Users\dengl\Anaconda3\envs\3.7\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dengl\Anaconda3\envs\3.7\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 6. Improve your model
Use grid search to find better parameters. 

In [27]:
parameters = parameters = {
    'clf__estimator__max_depth': [15, 25],
    'clf__estimator__n_estimators': [100,200]
        }
# create grid search object
cv = GridSearchCV(pipeline, param_grid=parameters)

In [28]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)
print(classification_report(y_test, y_pred, zero_division=1, target_names=y.columns))  # target_names=target_names

                        precision    recall  f1-score   support

               related       0.76      1.00      0.86      4991
               request       1.00      0.00      0.00      1158
                 offer       1.00      0.00      0.00        80
           aid_related       0.91      0.21      0.34      2717
          medical_help       1.00      0.00      0.00       582
      medical_products       1.00      0.00      0.00       391
     search_and_rescue       1.00      0.00      0.00       206
              security       1.00      0.00      0.00       163
              military       1.00      0.00      0.00       247
           child_alone       1.00      0.00      0.00        47
                 water       1.00      0.00      0.00       465
                  food       1.00      0.00      0.00       792
               shelter       0.00      0.00      0.00       628
              clothing       1.00      0.00      0.00       145
                 money       1.00      

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [29]:
from custom_transformer import StartingVerbExtractor
from sklearn.pipeline import FeatureUnion

ModuleNotFoundError: No module named 'custom_transformer'

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.